# DSPy + GEPA Tutorial
**Learning System Group, Oak Ridge National Labs**

**By Zachary L. Johnson-Scott**



This notebook walks through:
- Programming with DSPy modules and signatures
- Evaluating outputs with a custom metric
- Optimizing prompts with GEPA

We start with a single confusing passage, then scale to a small benchmark pack.


# Programming
**DSPy is a declarative open-source framework for building modular AI software, led by Standford**
* Language Models are engines/brains with unique attributes and behaviors 
* Modules help you describe AI behavior as code, not text or string
* Module are the strategy for invoking the foundational model behavior
* For every AI component in your system you define input/output behavior as a signature
* Signature are structured instruction in the form of a schema

* Signature Example: 

        question -> answer: list[int]; tells the system the output should be a list of ints
* Module Example

        class extract_int_list(dspy.Signature):

            """Extract a structured list of ints from the text.""" 

            text: str = dspy.InputField(desc="Receipts from project charges") 

            charges: list[int] = dspy.OutputField(desc="Extract total charges to project over the past six months")

        module = dspy.ChainOfThought(extract_int_class) 
        text = "text text text"

        response = module(text=text)


# Tutorial Setup / DSPy Setup

**Dependencies**

If you're running this locally, uncomment and run the install cell below.


In [ ]:
#Uncomment and run the following lines to install required packages

"""!pip install dspy
!pip install "slang[all]"
!pip install flashinfer -i https://flashinfer.ai/whl/cu121/torch2.4/"""

**Imports**

Keep imports in one place so later cells are easier to follow.


In [1]:
import os
import re
import random
import numpy as np
import dspy
from dspy.teleprompt.gepa.gepa import ScoreWithFeedback #Teleprompt is form of automatic prompt engineering (APE)


**DSPy Config**

Configure the task LLM (for normal predictions) and the reflection LLM (used by GEPA to propose prompt improvements).


In [2]:
#For local deployment on a virtual machine using DSPy

#Curl the virtual machine and model you want to use.
# Example command: curl http://medz2.ornl.gov:8200/v1


os.environ["OPENAI_API_BASE"] = "http://medz2.ornl.gov:8200/v1"  #Ensure it matches the server address
os.environ["OPENAI_API_KEY"] = "local"

task_lm = dspy.LM(model="openai/openai/gpt-oss-120b", #Ensure it matches the model you set up on the server
             api_base="http://medz2.ornl.gov:8200/v1",  #Ensure it matches the server address
             api_key="local", model_type="chat", temperature=0.1, max_tokens=50000)

reflection_lm = dspy.LM(
    model="openai/openai/gpt-oss-120b",      # can be the same model; give it higher temp
    temperature=0.9,
    max_tokens=5000
)

dspy.configure(lm=task_lm)


**Quick Sanity Check**

Confirm the LM is wired correctly with a tiny DSPy program.


In [3]:
#Test a math problem

math = dspy.ChainOfThought("question -> answer: float")
math(question="Two dice are tossed. What is the probability that the sum equals two?")

Prediction(
    reasoning='When two fair six‑sided dice are tossed, each die can show any integer from 1 to 6, giving \\(6 \\times 6 = 36\\) equally likely ordered outcomes.  \nThe sum of the two dice equals 2 only in the single case where both dice show a 1 (i.e., the outcome (1,\u202f1)).  \nThus the probability is the number of favorable outcomes divided by the total number of outcomes:\n\\[\nP(\\text{sum}=2) = \\frac{1}{36} \\approx 0.027777777777777776.\n\\]',
    answer=0.027777777777777776
)

# Problem Setup

**We will extract the *physical* memristor crossbar size (WL×BL) from a deliberately confusing passage.** 



In [4]:
benchmark_text = """Across internal documentation, array “size” is reported inconsistently, alternately referring to physical line count, logical tiling, or controller-visible address space. Orientation is also viewpoint-dependent; several reports transpose dimensions without explicit notice, depending on whether the discussion follows layout, architecture, or firmware convention.

Multiple prototype tiles are referenced. One tile is repeatedly described as “192-class,” indicating compliance with a pitch envelope rather than an explicit geometry. Within this constraint, the circuit schematic shows a 64-by-512 array, while an architecture note instead describes the same structure as 512-by-64, reflecting a rotated orientation rather than a different implementation. Elsewhere, the design is described as comprising two 64-by-256 sections; this refers to an intermediate partitioning prior to physical stitching, not to the final fabricated matrix. Mentions of 128-by-256 associated with this tile correspond to logical subdivision and should not be interpreted as the underlying crossbar geometry.

Other tiles in the system are described using nominal or effective dimensions that conflate addressability, redundancy, and yield screening, which has led to inconsistent shorthand usage in presentations and emails. A later internal note attempts to normalize terminology, specifying that unless otherwise stated, reported crossbar size refers to the fabricated wordline-by-bitline matrix, independent of enablement, remapping, or controller exposure.

All dimensions in this document should be interpreted under that convention."""

In [5]:
answer = "64X512"

**Crossbar Prompt**




In [6]:
class ExtractCrossbarSizePrompt(dspy.Signature):
    "What is the memristor crossbar size?."
    
    text: str = dspy.InputField(desc="Text containing memristor prototype details.")
    crossbar_size: str = dspy.OutputField(desc="The physical memristor crossbar size (WL×BL).")
    Wordlines: int = dspy.OutputField(desc="Number of wordlines in the physical crossbar.")
    Bitlines: int = dspy.OutputField(desc="Number of bitlines in the physical crossbar.")
    confidence: float = dspy.OutputField(desc="Confidence in the extracted answer, between 0.0 and 1.0.")
    
module = dspy.ChainOfThought(ExtractCrossbarSizePrompt)
response = module(text=benchmark_text)
print(f'Output: {response.crossbar_size}')
print(f' confidence score: {response.confidence}')
print(f"Answer: {answer}")

Output: 64×512
 confidence score: 0.96
Answer: 64X512


**Addendum that purposely adds complexity to the benchmark text**

In [7]:
benchmark_text_v2 = """Across internal documentation, array “size” is reported inconsistently, alternately referring to physical line count, logical tiling, or controller-visible address space. Orientation is also viewpoint-dependent; several reports transpose dimensions without explicit notice, depending on whether the discussion follows layout, architecture, or firmware convention.

Multiple prototype tiles are referenced. One tile is repeatedly described as “192-class,” indicating compliance with a pitch envelope rather than an explicit geometry. Within this constraint, the circuit schematic shows a 64-by-512 array, while an architecture note instead describes the same structure as 512-by-64, reflecting a rotated orientation rather than a different implementation. Elsewhere, the design is described as comprising two 64-by-256 sections; this refers to an intermediate partitioning prior to physical stitching, not to the final fabricated matrix. Mentions of 128-by-256 associated with this tile correspond to logical subdivision and should not be interpreted as the underlying crossbar geometry.

Other tiles in the system are described using nominal or effective dimensions that conflate addressability, redundancy, and yield screening, which has led to inconsistent shorthand usage in presentations and emails. A later internal note attempts to normalize terminology, specifying that unless otherwise stated, reported crossbar size refers to the fabricated wordline-by-bitline matrix, independent of enablement, remapping, or controller exposure.

All dimensions in this document should be interpreted under that convention. While the "192-class" tile follows the physical stitching of two $64 \times 256$ sections into a fabricated $64 \times 512$ matrix (often reported as $512 \times 64$ in architectural orientations), the newer Aegis-class tile utilizes a "quad-redundancy" layout.The Aegis-class is frequently referred to as a "512-nominal" array. However, this shorthand actually accounts for a $256 \times 256$ core that has been "virtually expanded" by the controller to mask row-repair sectors. Physical layouts for the Aegis show four quadrants, each $128 \times 128$. While the total bit count matches the 192-class logical subdivision ($128 \times 256$), the Aegis quadrants are stitched into a square $256 \times 256$ fabricated footprint to minimize signal latency.A common point of confusion arises in the "Yield Reports," which describe the Aegis as $128 \times 512$. This report refers to the "Post-Screening Address Map," where the $256 \times 256$ physical lines are remapped into a non-contiguous tall-and-narrow space for the firmware team."""

**Confidence decreases**
* GPT-OSS-120B still get the answer correct
* confidence score moves from 0.96 to 0.78

In [8]:
class ExtractCrossbarSizePrompt(dspy.Signature):
    "What is the memristor crossbar size?."
    
    text: str = dspy.InputField(desc="Text containing memristor prototype details.")
    crossbar_size: str = dspy.OutputField(desc="The physical memristor crossbar size (WL×BL).")
    Wordlines: int = dspy.OutputField(desc="Number of wordlines in the physical crossbar.")
    Bitlines: int = dspy.OutputField(desc="Number of bitlines in the physical crossbar.")
    confidence: float = dspy.OutputField(desc="Confidence in the extracted answer, between 0.0 and 1.0.")
    
module = dspy.ChainOfThought(ExtractCrossbarSizePrompt)
response = module(text=benchmark_text_v2)
print(f'Output: {response.crossbar_size}')
print(f' confidence score: {response.confidence}')
print(f"Answer: {answer}")

Output: 64×512
 confidence score: 0.78
Answer: 64X512


**Preferably we'd like the LLM to get the answer wrong to demonstrate clear improvement. However, this demonstrate the reliability of GPT-OSS-12OB**

# Zero Shot Test

In [10]:
import re
import math
import dspy
from dspy.teleprompt.gepa.gepa import ScoreWithFeedback

# ============================================================
# GOAL
#  A) Zero-shot test on NEW text (before GEPA)  -> baseline_generalization
#  B) GEPA optimize on TRAIN text               -> overfit/learned prompt
#  C) Test optimized on NEW benchmark set       -> generalization_after
#
# Target physical crossbar = 64×512 (WL×BL)
# ============================================================

# ------------------------------------------------------------
# 0) Provide your texts here
# ------------------------------------------------------------
# TRAIN text (what GEPA sees). You can keep your original "confusing prototype" text here.
TRAIN_TEXT = benchmark_text_v2  # you already defined this

# NEW zero-shot test set (unseen) to check generalization (add as many as you want)
# Put the text you just posted here as an example of an unseen benchmark.
TEST_TEXTS = [
    """Across internal documentation, array “size” is reported inconsistently, alternately referring to physical line count, logical tiling, or controller-visible address space. Orientation is also viewpoint-dependent; several reports transpose dimensions without explicit notice, depending on whether the discussion follows layout, architecture, or firmware convention.

Multiple prototype tiles are referenced. One tile is repeatedly described as “192-class,” indicating compliance with a pitch envelope rather than an explicit geometry. Within this constraint, the circuit schematic shows a 64-by-512 array, while an architecture note instead describes the same structure as 512-by-64, reflecting a rotated orientation rather than a different implementation. Elsewhere, the design is described as comprising two 64-by-256 sections; this refers to an intermediate partitioning prior to physical stitching, not to the final fabricated matrix. Mentions of 128-by-256 associated with this tile correspond to logical subdivision and should not be interpreted as the underlying crossbar geometry.

Other tiles in the system are described using nominal or effective dimensions that conflate addressability, redundancy, and yield screening, which has led to inconsistent shorthand usage in presentations and emails. A later internal note attempts to normalize terminology, specifying that unless otherwise stated, reported crossbar size refers to the fabricated wordline-by-bitline matrix, independent of enablement, remapping, or controller exposure.

All dimensions in this document should be interpreted under that convention. While the "192-class" tile follows the physical stitching of two $64 \\times 256$ sections into a fabricated $64 \\times 512$ matrix (often reported as $512 \\times 64$ in architectural orientations), the newer Aegis-class tile utilizes a "quad-redundancy" layout.The Aegis-class is frequently referred to as a "512-nominal" array. However, this shorthand actually accounts for a $256 \\times 256$ core that has been "virtually expanded" by the controller to mask row-repair sectors. Physical layouts for the Aegis show four quadrants, each $128 \\times 128$. While the total bit count matches the 192-class logical subdivision ($128 \\times 256$), the Aegis quadrants are stitched into a square $256 \\times 256$ fabricated footprint to minimize signal latency.A common point of confusion arises in the "Yield Reports," which describe the Aegis as $128 \\times 512$. This report refers to the "Post-Screening Address Map," where the $256 \\times 256$ physical lines are remapped into a non-contiguous tall-and-narrow space for the firmware team."""
]

# Gold target
GOLD_WL, GOLD_BL = 64, 512
answer = f"{GOLD_WL}×{GOLD_BL}"  # for printing

# ------------------------------------------------------------
# 1) Signature + program
# ------------------------------------------------------------
class ExtractCrossbarSizePrompt(dspy.Signature):
    "What is the memristor crossbar size?."
    text: str = dspy.InputField(desc="Text containing memristor prototype details.")
    crossbar_size: str = dspy.OutputField(desc="The physical memristor crossbar size (WL×BL).")
    Wordlines: int = dspy.OutputField(desc="Number of wordlines in the physical crossbar.")
    Bitlines: int = dspy.OutputField(desc="Number of bitlines in the physical crossbar.")
    confidence: float = dspy.OutputField(desc="Confidence in the extracted answer, between 0.0 and 1.0.")

student = dspy.ChainOfThought(ExtractCrossbarSizePrompt)

# ------------------------------------------------------------
# 2) Helpers
# ------------------------------------------------------------
def clamp01(x):
    try:
        x = float(x)
    except Exception:
        return 0.0
    if math.isnan(x) or math.isinf(x):
        return 0.0
    return max(0.0, min(1.0, x))

def parse_dims(s: str):
    if not s:
        return None
    s = str(s).replace("×", "x").replace("X", "x")
    m = re.search(r"(\d+)\s*x\s*(\d+)", s)
    if not m:
        return None
    return int(m.group(1)), int(m.group(2))

def get_pred_dims(pred):
    wl = getattr(pred, "Wordlines", None)
    bl = getattr(pred, "Bitlines", None)
    if isinstance(wl, int) and isinstance(bl, int) and wl > 0 and bl > 0:
        return wl, bl

    dims = parse_dims(getattr(pred, "crossbar_size", "") or "")
    if dims:
        return dims

    return parse_dims(str(pred))

def confidence_nudge(conf: float) -> str:
    conf = clamp01(conf)
    if conf < 0.8:
        return (
            "LOW_CONFIDENCE. Re-check extraction carefully:\n"
            "- Verify WL×BL refers to the physical fabricated crossbar (not enabled/effective/class/tiled).\n"
            "- Watch for transposed conventions (BL×WL) and 'rotated' diagrams.\n"
            "- Ensure Wordlines/Bitlines are positive integers and consistent with crossbar_size.\n"
            "- If uncertain, locate the most explicit 'physical/fabricated' sentence and use that."
        )
    else:
        return (
            "HIGH_CONFIDENCE. Still do a quick sanity check:\n"
            "- Confirm WL×BL order and no transpose.\n"
            "- Confirm you didn't pick enabled/effective/class sizes."
        )

# ------------------------------------------------------------
# 3) Metric: confidence-only + tiny guardrails
# ------------------------------------------------------------
def confidence_only_metric(gold, pred, trace=None, pred_name=None, pred_trace=None):
    conf = clamp01(getattr(pred, "confidence", 0.0))

    # Guardrail A: must output parseable dimensions somewhere
    dims = get_pred_dims(pred)
    if not dims:
        return ScoreWithFeedback(
            score=0.0,
            feedback="FAIL:NO_PARSE_DIMENSIONS. " + confidence_nudge(conf)
        )

    # Guardrail B: if numeric fields exist and crossbar_size is parseable, they must match
    wl = getattr(pred, "Wordlines", None)
    bl = getattr(pred, "Bitlines", None)
    parsed_from_str = parse_dims(getattr(pred, "crossbar_size", "") or "")
    if isinstance(wl, int) and isinstance(bl, int) and parsed_from_str:
        if (wl, bl) != parsed_from_str:
            return ScoreWithFeedback(
                score=0.0,
                feedback="FAIL:INCONSISTENT_FIELDS. " + confidence_nudge(conf)
            )

    return ScoreWithFeedback(score=conf, feedback=confidence_nudge(conf))

# ------------------------------------------------------------
# 4) Correctness check (NOT used by GEPA, used for reporting generalization)
# ------------------------------------------------------------
def correctness(pred, gold_wl=GOLD_WL, gold_bl=GOLD_BL):
    dims = get_pred_dims(pred)
    return float(dims == (gold_wl, gold_bl))

# ------------------------------------------------------------
# 5) A helper runner to evaluate a program on a list of texts
# ------------------------------------------------------------
def eval_on_texts(program, texts, label=""):
    results = []
    correct = 0
    for i, t in enumerate(texts):
        pred = program(text=t)
        dims = get_pred_dims(pred)
        conf = clamp01(getattr(pred, "confidence", 0.0))
        is_correct = correctness(pred)
        correct += int(is_correct)

        results.append((i, dims, conf, is_correct, getattr(pred, "crossbar_size", None)))

        print("\n" + "="*80)
        print(f"{label} EX {i}")
        print("crossbar_size:", getattr(pred, "crossbar_size", None))
        print("Wordlines:", getattr(pred, "Wordlines", None), "Bitlines:", getattr(pred, "Bitlines", None))
        print("parsed dims:", dims)
        print("confidence:", conf)
        print("correct:", bool(is_correct))
        print("gold:", f"{GOLD_WL}×{GOLD_BL} (WL×BL)")

    acc = correct / max(1, len(texts))
    print("\n" + "-"*80)
    print(f"{label} accuracy on {len(texts)} texts: {correct}/{len(texts)} = {acc:.3f}")
    return results, acc

# ============================================================
# A) ZERO-SHOT GENERALIZATION TEST (before any optimization)
# ============================================================
print("\n\n### ZERO-SHOT TEST (student on unseen TEST_TEXTS) ###")
_ = eval_on_texts(student, TEST_TEXTS, label="STUDENT ZERO-SHOT")

# ============================================================
# B) GEPA OPTIMIZATION on TRAIN (single example)
# ============================================================
trainset = [
    dspy.Example(text=TRAIN_TEXT, gold_answer=answer).with_inputs("text")
]
valset = trainset  # demo/inference-time scaling; for real generalization use a separate valset

baseline_pred = student(text=TRAIN_TEXT)
baseline_score = confidence_only_metric(trainset[0], baseline_pred)

print("\n\n### TRAIN BASELINE (student on TRAIN_TEXT) ###")
print("crossbar_size:", getattr(baseline_pred, "crossbar_size", None))
print("Wordlines:", getattr(baseline_pred, "Wordlines", None), "Bitlines:", getattr(baseline_pred, "Bitlines", None))
print("confidence:", clamp01(getattr(baseline_pred, "confidence", 0.0)))
print("metric score:", float(baseline_score.score) if hasattr(baseline_score, "score") else float(baseline_score))
print("metric feedback:", getattr(baseline_score, "feedback", None))
print("sanity correctness (not used by GEPA):", correctness(baseline_pred))

gepa = dspy.GEPA(
    metric=confidence_only_metric,
    reflection_lm=reflection_lm,   # you already set this up
    auto="light",
    seed=0,
    track_stats=True,
    track_best_outputs=True,
)

optimized = gepa.compile(student, trainset=trainset, valset=valset)

# Optional: print the optimized instructions
print("\n\n### OPTIMIZED INSTRUCTIONS (what GEPA changed) ###")
try:
    print(optimized.predict.signature.instructions)
except Exception:
    print("Could not access optimized.predict.signature.instructions (module layout differs).")

# ============================================================
# C) POST-OPT GENERALIZATION TEST (optimized on unseen TEST_TEXTS)
# ============================================================
print("\n\n### POST-OPT ZERO-SHOT TEST (optimized on unseen TEST_TEXTS) ###")
_ = eval_on_texts(optimized, TEST_TEXTS, label="OPTIMIZED ZERO-SHOT")

# Also show optimized behavior on TRAIN (it will usually be very high confidence)
opt_train_pred = optimized(text=TRAIN_TEXT)
opt_train_score = confidence_only_metric(trainset[0], opt_train_pred)
print("\n\n### OPTIMIZED on TRAIN_TEXT (for reference) ###")
print("crossbar_size:", getattr(opt_train_pred, "crossbar_size", None))
print("Wordlines:", getattr(opt_train_pred, "Wordlines", None), "Bitlines:", getattr(opt_train_pred, "Bitlines", None))
print("confidence:", clamp01(getattr(opt_train_pred, "confidence", 0.0)))
print("metric score:", float(opt_train_score.score) if hasattr(opt_train_score, "score") else float(opt_train_score))
print("sanity correctness (not used by GEPA):", correctness(opt_train_pred))
print("Gold:", answer)




### ZERO-SHOT TEST (student on unseen TEST_TEXTS) ###


2026/02/06 16:49:50 INFO dspy.teleprompt.gepa.gepa: Running GEPA for approx 384 metric calls of the program. This amounts to 192.00 full evals on the train+val set.
2026/02/06 16:49:50 INFO dspy.teleprompt.gepa.gepa: Using 1 examples for tracking Pareto scores. You can consider using a smaller sample of the valset to allow GEPA to explore more diverse solutions within the same budget. GEPA requires you to provide the smallest valset that is just large enough to match your downstream task distribution, while providing as large trainset as possible.



STUDENT ZERO-SHOT EX 0
crossbar_size: 64×512
Wordlines: 64 Bitlines: 512
parsed dims: (64, 512)
confidence: 0.78
correct: True
gold: 64×512 (WL×BL)

--------------------------------------------------------------------------------
STUDENT ZERO-SHOT accuracy on 1 texts: 1/1 = 1.000


### TRAIN BASELINE (student on TRAIN_TEXT) ###
crossbar_size: 64×512
Wordlines: 64 Bitlines: 512
confidence: 0.78
metric score: 0.78
metric feedback: LOW_CONFIDENCE. Re-check extraction carefully:
- Verify WL×BL refers to the physical fabricated crossbar (not enabled/effective/class/tiled).
- Watch for transposed conventions (BL×WL) and 'rotated' diagrams.
- Ensure Wordlines/Bitlines are positive integers and consistent with crossbar_size.
- If uncertain, locate the most explicit 'physical/fabricated' sentence and use that.
sanity correctness (not used by GEPA): 1.0


GEPA Optimization:   0%|          | 0/384 [00:00<?, ?rollouts/s]2026/02/06 16:49:50 INFO dspy.evaluate.evaluate: Average Metric: 0.78 / 1 (78.0%)
2026/02/06 16:49:50 INFO dspy.teleprompt.gepa.gepa: Iteration 0: Base program full valset score: 0.78 over 1 / 1 examples
2026/02/06 16:49:50 INFO dspy.teleprompt.gepa.gepa: Iteration 1: Selected program 0 score: 0.78


Average Metric: 2.34 / 3 (78.0%): 100%|██████████| 3/3 [00:00<00:00, 1652.17it/s]

...

2026/02/06 16:49:52 INFO dspy.evaluate.evaluate: Average Metric: 2.8499999999999996 / 3 (95.0%)
2026/02/06 16:49:52 INFO dspy.teleprompt.gepa.gepa: Iteration 63: New subsample score 2.8499999999999996 is not better than old score 2.8499999999999996, skipping
2026/02/06 16:49:52 INFO dspy.teleprompt.gepa.gepa: Iteration 64: Selected program 2 score: 0.95


Average Metric: 2.85 / 3 (95.0%): 100%|██████████| 3/3 [00:00<00:00, 1928.12it/s]

2026/02/06 16:49:52 INFO dspy.evaluate.evaluate: Average Metric: 2.8499999999999996 / 3 (95.0%)
2026/02/06 16:49:52 INFO dspy.teleprompt.gepa.gepa: Iteration 64: Proposed new text for predict: **Task Overview – Extract the Physical (Fabricated) Cross‑Bar Geometry**

You will be given a *single paragraph* (or a short passage) that talks about one or more memristor cross‑bar arrays.  
Your job is to determine the **actual, fabricated matrix size** that was laid out on silicon – i.e., the real number of **wordlines (WL)** and **bitlines (BL)** that exist on the die – and report it in a strict, structured format.

---

### 1. What “physical / fabricated” means
- **Physical (fabricated) geometry** = the literal WL‑by‑BL matrix that was drawn on the die and manufactured.
- **Never use** numbers that are described as:
  - logical, virtual, nominal, “enabled”, “effective”, “post‑screening”, “address‑map”, “controller‑visible”, or any other indirect count.
  - dimensions of a *sub‑tile*, *redun

2026/02/06 16:49:52 INFO dspy.evaluate.evaluate: Average Metric: 2.8499999999999996 / 3 (95.0%)
2026/02/06 16:49:52 INFO dspy.teleprompt.gepa.gepa: Iteration 64: New subsample score 2.8499999999999996 is not better than old score 2.8499999999999996, skipping
GEPA Optimization:  99%|█████████▉| 381/384 [00:02<00:00, 181.70rollouts/s]




### OPTIMIZED INSTRUCTIONS (what GEPA changed) ###
**Task: Extract the Physical (Fabricated) Cross‑Bar Geometry**

You will receive a single paragraph (or a short passage) that describes one or more memristor cross‑bar arrays.  
Your job is to determine the **actual, fabricated matrix size** – the number of wordlines (WL) and bitlines (BL) that exist on the silicon die – and present the result in a strict, structured format.

---

### 1. What “physical / fabricated” means
- **Physical (fabricated) geometry** = the literal wordline‑by‑bitline matrix that was laid out on the die.  
- **Do NOT use** any of the following as the final size:
  * Logical, virtual, nominal, “enabled”, “effective”, “post‑screening”, “address‑map”, or controller‑visible dimensions.  
  * Sizes that refer only to a sub‑tile, redundancy scheme, or a partition that is later stitched together unless the text explicitly says that the stitched result *is* the fabricated matrix.  
  * Any size that is described as “v

In [12]:
print(optimized.predict.signature.instructions)


**Task: Extract the Physical (Fabricated) Cross‑Bar Geometry**

You will receive a single paragraph (or a short passage) that describes one or more memristor cross‑bar arrays.  
Your job is to determine the **actual, fabricated matrix size** – the number of wordlines (WL) and bitlines (BL) that exist on the silicon die – and present the result in a strict, structured format.

---

### 1. What “physical / fabricated” means
- **Physical (fabricated) geometry** = the literal wordline‑by‑bitline matrix that was laid out on the die.  
- **Do NOT use** any of the following as the final size:
  * Logical, virtual, nominal, “enabled”, “effective”, “post‑screening”, “address‑map”, or controller‑visible dimensions.  
  * Sizes that refer only to a sub‑tile, redundancy scheme, or a partition that is later stitched together unless the text explicitly says that the stitched result *is* the fabricated matrix.  
  * Any size that is described as “virtual expansion”, “quad‑redundancy layout”, “nominal

# Evaluation

We build a metric that checks the parsed WL×BL, penalizes transposes, and provides feedback to GEPA.


**First Prompt Proposed via Reflection**

dspy.teleprompt.gepa.gepa: Iteration 1: Proposed new text for predict: **Task Overview**
You will be given a free‑form text that describes one or more memristor cross‑bar prototypes.  
The text intentionally mixes several different notions of “size”:
* **Physical cross‑bar size** – the actual fabricated number of wordlines (WL) and bitlines (BL).  
* **Enabled/usable size** – a subset that survives yield screening.  
* **Logical or effective size** – the address space the controller presents after remapping, tiling, or adding spare lines.  
* **Class or nominal descriptors** – vague references such as “192×192‑class”, “128 crossbar”, “1k×1k”, etc.

Your job is to extract **the single physical cross‑bar size** that is explicitly stated as the fabricated dimensions of the final stitched array.  
You must ignore every other number that refers to logical partitions, spares, enabled subsets, class budgets, or alternative orientation (e.g., “bitlines × wordlines”).  

**What to Output**
Produce three fields in the exact order and format below:

```
### crossbar_size
<WL>×<BL>

### Wordlines
<WL>

### Bitlines
<BL>

**How GEPA Uses the Metric**

GEPA treats the metric as the *training signal*. When the model is wrong, the metric returns a score plus feedback.
GEPA then asks a reflection model to propose prompt improvements that would raise that score.
This is why the feedback text is specific about *physical WL×BL*, transposes, and common distractors.


**How does it know we are trying to confuse it?**

**Bottom line**

It “knows you’re trying to confuse it” because:

* Passage literally says it’s confusing

* GEPA is explicitly designed to reflect on failures and propose instructions

* Used train=val on a single item, which is basically “find the perfect prompt for this one task.”